Прежде чем запускать следующее, нужно пройти VK-авторизацию!

In [1]:
%%javascript
var kernel = IPython.notebook.kernel;
var command = "currentUrl = " + "'" + window.location + "'";
kernel.execute(command);

<IPython.core.display.Javascript object>

In [2]:
from urlparse import urlparse

r = urlparse(currentUrl)
code = r.query[5:]

url = ("https://oauth.vk.com/access_token"
       "?client_id=5429181" 
       "&client_secret=uAZqxwa2hgjLa4KLvbTG"
       "&code=" + code +
       "&redirect_uri="  + r.scheme + "://" + r.netloc + r.path)

In [3]:
import json
import urllib2   

data = json.load(urllib2.urlopen(url))
accessToken = data['access_token']

# Достаем участников группы

In [36]:
from urllib import urlencode
import time

groups = ["109490789", 
          "2100831", 
          "11912933", 
          "1167943", 
          "15335079", 
          "12813079", 
          "68478644",
          "16vzvoduvo",
          "59565067",
          "61511779",
          "1182973",
          "1119648",
          "21754234",
          "14294712",
          "7591998",
          "8702762",
          "23386497",
          "1566321",
          "55523638"]

users = []

for group in groups:
    params = {"group_id": group, 
              "count": 200, 
              "fields": "sex,education",
              "access_token": accessToken}
    params = urlencode(params)

    url = "https://api.vk.com/method/groups.getMembers?" + params
    data = json.load(urllib2.urlopen(url))
    if data.get('response', ""): 
        data_users = data['response']['users']
    
        print("extracted %d users from %s" % (len(data_users), group))
        users.extend(data_users)
        time.sleep(0.3)
    
print("sum: %d" % len(users))

extracted 26 users from 109490789
extracted 200 users from 2100831
extracted 51 users from 11912933
extracted 104 users from 1167943
extracted 41 users from 15335079
extracted 31 users from 12813079
extracted 17 users from 68478644
extracted 13 users from 16vzvoduvo
extracted 97 users from 59565067
extracted 30 users from 61511779
extracted 200 users from 1182973
extracted 200 users from 1119648
extracted 91 users from 21754234
extracted 8 users from 14294712
extracted 35 users from 7591998
extracted 34 users from 8702762
extracted 2 users from 23386497
extracted 86 users from 1566321
extracted 44 users from 55523638
sum: 1310


# Достаем у всех найденных пользователей персональную информацию

In [50]:
import json

#    for user in users: 
#        print(user.get('personal', ""))

uids = []        

#for i in range(0,20):
#    vyborka.append(users[i].get('uid', ""))
for user in users:
    uids.append(user.get('uid', ""))

part = uids[:100]
    
#print(uids)
#print(vyborka)
pers = []
params = {"user_ids": part, 
          "fields": "personal",
          "access_token": accessToken}
params = urlencode(params)

url = "https://api.vk.com/method/users.get?" + params
data = json.load(urllib2.urlopen(url))
pers_data_array = data['response']

print(pers_data_array)

with open('data_personal.json', 'w') as outfile:
    json.dump(pers_data_array, outfile)
    

    

[{u'personal': {u'religion': u'Nothingdoing', u'alcohol': 1, u'smoking': 1, u'inspired_by': u'\u043a\u0443\u0440\u0435\u043d\u0438\u0435 \u0438 \u0430\u043b\u043a\u043e\u0433\u043e\u043b\u044c'}, u'first_name': u'Mega', u'last_name': u'\u0411\u044b\u0434\u043b\u043e\u0432\u0438\u0447', u'uid': 378807}, {u'first_name': u'\u041d\u0430\u0440\u0438\u043d\u0430', u'last_name': u'\u0421\u0442\u0435\u043f\u0430\u043d\u044f\u043d\u0446', u'uid': 545614}, {u'personal': {u'people_main': 2, u'alcohol': 5, u'inspired_by': u'\u0434\u043e\u0447\u044c)))', u'political': 3, u'religion': u'\u0421\u0432\u0435\u0442\u0441\u043a\u0438\u0439 \u0433\u0443\u043c\u0430\u043d\u0438\u0437\u043c', u'smoking': 5, u'life_main': 1}, u'first_name': u'\u0421\u0442\u0430\u043d\u0438\u0441\u043b\u0430\u0432', u'last_name': u'\u041a\u043e\u0437\u043b\u043e\u0432', u'uid': 661576}, {u'personal': [], u'first_name': u'\u0422\u0451\u043c\u043d\u044b\u0439', u'last_name': u'\u0418\u043d\u043a\u0432\u0438\u0437\u0438\u0442\u0

# Построим графиченьки по перс информации

In [16]:
import numpy as np
import pandas as pd
import matplotlib
from collections import defaultdict
from matplotlib import pyplot as plt